# EXTRACCION, TRANSFORMACIÓN Y LIMPIEZA 

## Base de datos: Movies

Se utiliza la libreria de pandas para dar inicio a la estracción y tratamiento de los datos.

In [63]:
import pandas as pd
import numpy as np
import json
import ast
import os

Inicio la extracción, primero del detaset "movies" y luego del dataset "credits" para despues unirlos. 

In [64]:
# Extraigo el dataset "movies" y reviso el tamaño
df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
df_movies.shape

C:\Users\anavi\AppData\Local\Temp\ipykernel_18280\131553651.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')


(45466, 24)

In [65]:
# Extraigo los dataset "credits" y reviso el tamaño
df_credits = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')
df_credits.shape

(45476, 3)

In [66]:
# Quiero saber el tamaño de mi dataset para reducirlo hasta llegar a 100 megas
tamaño1 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
tamaño2 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')

print(f"Tamaño movies: {tamaño1 / (1024 * 1024):.2f} MB y tamaño credits: {tamaño2 / (1024 * 1024):.2f} MB")

Tamaño movies: 32.85 MB y tamaño credits: 181.12 MB


Como veo que el dataset credits es mucho más grande que el dataset movies entonces empiezo limpiandolo para reducirlo al máximo.

In [67]:
df_credits[df_credits.duplicated(keep=False)].shape # Verifico si hay datos duplicados para eliminarlos posteriormente

(73, 3)

In [68]:
df_credits = df_credits.drop_duplicates() # Elimino los datos duplicados en mi dataset original
df_credits[df_credits.duplicated(keep=False)].shape #Verifico que hayan quedado eliminados

(0, 3)

In [69]:
type(df_credits['cast'][0]) # se verifica el tipo de datos, deben ser listas o dicccionarios para poder normalizarlos

str

In [70]:
#Se convierte el tipo de datos de String a Diccionario o lista

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_credits['cast'] = df_credits['cast'].apply(convertir_a_diccionario)

type(df_credits['cast'][0])

list

In [71]:
# Se muestran los tipos de datos únicos en la columnas

df_credits['cast'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [72]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_credits.iterrows():
    entry = row['cast']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_cast = pd.concat(expanded_dfs, ignore_index=True)

In [73]:
df_cast.columns #Para verificar las columnas existentes en el nuevo data frame

Index(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order',
       'profile_path'],
      dtype='object')

In [74]:
df_cast.shape #Para verificar el tamaño del nuevo data frame

(562132, 8)

In [75]:
df_cast = df_cast[['id'] + [col for col in df_cast.columns if col != 'id']] #Dejo de primera la columna "id"
df_cast = df_cast.drop(columns=['credit_id', 'gender', 'cast_id', 'profile_path']) #Elimino las columnas innecesarias

In [82]:
df_cast = df_cast[df_cast['order'].between(1, 5)] # Filtrar los primeros 5 actores y eliminar el resto

In [83]:
df_cast.head(20)

,id,cast_id,character,name,order
0,862,14.0,Woody (voice),Tom Hanks,1
1,862,15.0,Buzz Lightyear (voice),Tim Allen,2
2,862,16.0,Mr. Potato Head (voice),Don Rickles,3
3,862,17.0,Slinky Dog (voice),Jim Varney,4
4,862,18.0,Rex (voice),Wallace Shawn,5
13,8844,1.0,Alan Parrish,Robin Williams,1
14,8844,8.0,Samuel Alan Parrish / Van Pelt,Jonathan Hyde,2
15,8844,2.0,Judy Sheperd,Kirsten Dunst,3
16,8844,24.0,Peter Shepherd,Bradley Pierce,4
17,8844,10.0,Sarah Whittle,Bonnie Hunt,5


In [84]:
df_cast.shape #Para verificar el tamaño nuevamente

(202338, 5)

In [76]:
# Calcular el uso de memoria del DataFrame en bytes
uso_memoria = df_movies.memory_usage(deep=True).sum()

# Convertir el uso de memoria a megabytes
uso_memoria_mb = uso_memoria / (1024 * 1024)

print(f"El DataFrame utiliza aproximadamente {uso_memoria_mb:.2f} MB de memoria.")


El DataFrame utiliza aproximadamente 79.46 MB de memoria.
